In [7]:
cd character-extraction-master

C:\Users\Dell\downloads\character-extraction-master\character-extraction-master


In [39]:
import json
import nltk
import re

from collections import defaultdict
from nltk.corpus import stopwords
from pattern.en import parse, Sentence, mood
from pattern.db import csv
from pattern.vector import Document, NB

In [111]:
import re
text=text.replace("Harry Potter and the Philosophers Stone - J.K."," ")
text=text.replace("Page |"," ")
text=text.replace("Page"," ")
text=text.replace("Rowling"," ")
text = re.sub(r'[0-9]+', '', text)
text=re.sub('\s+',' ',text)

with open("hp_ps_co2.txt", "w",encoding="utf-8") as f:
    f.write(text)
    f.close()

In [9]:
def readText():
    """
    Reads the text from a text file.
    """
    with open("hp_ps_co2.txt", "rb") as f:
        text = f.read().decode('utf-8-sig')
    return text

In [10]:
def chunkSentences(text):
    """
    Parses text into parts of speech tagged with parts of speech labels.

    Used for reference: https://gist.github.com/onyxfish/322906
    """
    sentences = nltk.sent_tokenize(text)
    tokenizedSentences = [nltk.word_tokenize(sentence)
                          for sentence in sentences]
    taggedSentences = [nltk.pos_tag(sentence)
                       for sentence in tokenizedSentences]
    if nltk.__version__[0:2] == "2.":
        chunkedSentences = nltk.batch_ne_chunk(taggedSentences, binary=True)
    else:
        chunkedSentences = nltk.ne_chunk_sents(taggedSentences, binary=True)
    return chunkedSentences

In [12]:
import nltk

In [13]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [14]:
def extractEntityNames(tree, _entityNames=None):
    """
    Creates a local list to hold nodes of tree passed through, extracting named
    entities from the chunked sentences.

    Used for reference: https://gist.github.com/onyxfish/322906
    """
    if _entityNames is None:
        _entityNames = []
    try:
        if nltk.__version__[0:2] == "2.":
            label = tree.node
        else:
            label = tree.label()
    except AttributeError:
        pass
    else:
        if label == 'NE':
            _entityNames.append(' '.join([child[0] for child in tree]))
        else:
            for child in tree:
                extractEntityNames(child, _entityNames=_entityNames)
    return _entityNames

In [15]:
def buildDict(chunkedSentences, _entityNames=None):
    """
    Uses the global entity list, creating a new dictionary with the properties
    extended by the local list, without overwriting.

    Used for reference: https://gist.github.com/onyxfish/322906
    """
    if _entityNames is None:
        _entityNames = []

    for tree in chunkedSentences:
        extractEntityNames(tree, _entityNames=_entityNames)

    return _entityNames

In [16]:
nltk.download('maxent_ne_chunker')

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!


True

In [17]:
def removeStopwords(entityNames, customStopWords=None):
    """
    Brings in stopwords and custom stopwords to filter mismatches out.
    """
    # Memoize custom stop words
    if customStopWords is None:
        with open("customStopWords.txt", "rb") as f:
            customStopwords = f.read().split(', ')

    for name in entityNames:
        if name in stopwords.words('english') or name in customStopwords:
            entityNames.remove(name)

In [18]:
nltk.download('words')

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [19]:
def getMajorCharacters(entityNames):
    """
    Adds names to the major character list if they appear frequently.
    """
    return {name for name in entityNames if entityNames.count(name) > 15}

In [242]:
def getMajorCharacters(entityNames):
    """
    Adds names to the major character list if they appear frequently.
    """
    return {name for name in entityNames if entityNames.count(name) > 50}

In [20]:
def splitIntoSentences(text):
    """
    Split sentences on .?! "" and not on abbreviations of titles.
    Used for reference: http://stackoverflow.com/a/8466725
    """
    sentenceEnders = re.compile(r"""
    # Split sentences on whitespace between them.
    (?:               # Group for two positive lookbehinds.
      (?<=[.!?])      # Either an end of sentence punct,
    | (?<=[.!?]['"])  # or end of sentence punct and quote.
    )                 # End group of two positive lookbehinds.
    (?<!  Mr\.   )    # Don't end sentence on "Mr."
    (?<!  Mrs\.  )    # Don't end sentence on "Mrs."
    (?<!  Ms\.   )    # Don't end sentence on "Ms."
    (?<!  Jr\.   )    # Don't end sentence on "Jr."
    (?<!  Dr\.   )    # Don't end sentence on "Dr."
    (?<!  Prof\. )    # Don't end sentence on "Prof."
    (?<!  Sr\.   )    # Don't end sentence on "Sr."
    \s+               # Split on whitespace between sentences.
    """, re.IGNORECASE | re.VERBOSE)
    return sentenceEnders.split(text)

In [21]:
def compareLists(sentenceList, majorCharacters):
    """
    Compares the list of sentences with the character names and returns
    sentences that include names.
    """
    characterSentences = defaultdict(list)
    for sentence in sentenceList:
        for name in majorCharacters:
            if re.search(r"\b(?=\w)%s\b(?!\w)" % re.escape(name),
                         sentence,
                         re.IGNORECASE):
                characterSentences[name].append(sentence)
    return characterSentences

In [22]:
def extractMood(characterSentences):
    """
    Analyzes the sentence using grammatical mood module from pattern.
    """
    characterMoods = defaultdict(list)
    for key, value in characterSentences.items():
        for x in value:
            characterMoods[key].append(mood(Sentence(parse(str(x),
                                                           lemmata=True))))
    return characterMoods

In [246]:
def extractSentiment(characterSentences):
    """
    Trains a Naive Bayes classifier object with the reviews.csv file, analyzes
    the sentence, and returns the tone.
    """
    nb = NB()
    characterTones = defaultdict(list)
    for review, rating in csv("reviews.csv"):
        nb.train(Document(review, type=int(rating), stopwords=True))
    for key, value in characterSentences.items():
        for x in value:
            characterTones[key].append(nb.classify(str(x)))
    return characterTones

In [19]:
def writeAnalysis(sentenceAnalysis):
    """
    Writes the sentence analysis to a text file in the same directory.
    """
    with open("sentenceAnalysis.txt", "wb") as f:
        for item in sentenceAnalysis.items():
            f.write("%s:%s\n" % item)

In [20]:
def writeToJSON(sentenceAnalysis):
    """
    Writes the sentence analysis to a JSON file in the same directory.
    """
    with open("sentenceAnalysis.json", "wb") as f:
        json.dump(sentenceAnalysis, f)

In [23]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [248]:
!pip install many_stop_words

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [24]:
from many_stop_words import get_stop_words
stop_list=list(get_stop_words('en'))        #About 900 stop words
nltk_words = list(stopwords.words('english'))   #About 150 stop words
stop_list.extend(nltk_words)
stop_list += ["Yeh"]

In [25]:
text = readText()

In [26]:
chunkedSentences = chunkSentences(text)

In [27]:
entityNames = buildDict(chunkedSentences)

In [28]:
entNames = list(set(entityNames))
len(entNames)

810

In [29]:
 majorCharacters = getMajorCharacters(entityNames)
len(majorCharacters)
majCharacters=list(majorCharacters)

In [30]:
majorCharacters

{'Albus Dumbledore',
 'Aunt Petunia',
 'Bane',
 'Crabbe',
 'Dudley',
 'Dumbledore',
 'Dursleys',
 'Fang',
 'Fat Lady',
 'Filch',
 'Fluffy',
 'Fred',
 'George',
 'George Weasley',
 'Goyle',
 'Great Hall',
 'Gringotts',
 'Gryffindor',
 'Hagrid',
 'Harry',
 'Harry Harry',
 'Harry Potter',
 'Hedwig',
 'Hermione',
 'Hermione Granger',
 'Hogwarts',
 'House',
 'Lord Voldemort',
 'Madam Hooch',
 'Madam Pomfrey',
 'Malfoy',
 'Mirror',
 'Mr. Dursley',
 'Mr. Ollivander',
 'Muggle',
 'Neville',
 'Nicolas Flamel',
 'Norbert',
 'Oliver Wood',
 'Peeves',
 'Percy',
 'Philosophers Stone',
 'Potter',
 'Privet Drive',
 'Professor',
 'Professor McGonagall',
 'Professor Snape',
 'Quaffle',
 'Quidditch',
 'Quirrell',
 'Ron',
 'Ronan',
 'Seamus',
 'Slytherin',
 'Snape',
 'Sorcerer',
 'Uncle Vernon',
 'Voldemort',
 'Weasley',
 'Wood',
 'Yeh'}

In [31]:
len(majorCharacters)

61

In [33]:
for name in majorCharacters:
        if name in stop_list:
            majorCharacters.remove(name)

In [36]:
import re

In [37]:
sentenceList = splitIntoSentences(text)

In [40]:
characterSentences = compareLists(sentenceList, majorCharacters)

In [137]:
characterMoods = defaultdict(list)
for key, value in characterSentences.items():
        for x in value:
            continue
            #characterMoods[key].append(mood(Sentence(parse(str(x)))))

In [260]:
sentenceAnalysis = defaultdict(list,
                                   [(k,characterSentences[k])
                                    for k in characterSentences])

In [41]:
with open("sentenceAnalysis_hp_co.txt", "w",encoding="utf-8") as f:
    for k in characterSentences:
        f.write("\n\n----------------------------------------------\n\n")
        f.write("\n*************************************************\n")
        f.write(k)
        f.write("\n*************************************************\n")
        f.write("\n----------------------------------------------\n")
        for x in characterSentences[k]:
            f.write(x)
            f.write("\n----------------------------------------------\n")

In [ ]:
#with open("sentenceAnalysis_hp.txt", "w") as f:
      # for item in sentenceAnalysis.items():
            #f.write("%s:%s\n" % item)

In [165]:
with open("sentenceAnalysis.json", "w") as f:
        json.dump(sentenceAnalysis, f)

In [42]:
import spacy
import en_core_web_sm
nlp = en_core_web_sm.load()

In [263]:
with open("charecter_freq2.txt", "w",encoding="utf-8") as f:
    for name in majorCharacters:
        f.write(name)
        f.write(":\n")
        f.write(str(entityNames.count(name)))
        f.write("\n--------------------------------------------------\n")

In [43]:
lines_list = open('pa.txt').read().splitlines()

In [264]:
type(lines_list[0])

str

In [44]:
characterDescription = defaultdict(set)

In [45]:
import re

In [46]:
def compareLists1(sentenceList1, majorCharacters1,k):
    """
    Compares the list of sentences with the character names and returns
    sentences that include names.
    """
    for sentence in sentenceList1:
        for name in majorCharacters1:
            if re.search(r"\b(?=\w)%s\b(?!\w)" % re.escape(name),sentence,re.IGNORECASE):
                characterDescription[k].add(sentence)

In [47]:
for k in characterSentences:
    compareLists1(characterSentences[k],lines_list,k)

In [48]:
z

In [49]:
for k in characterDescription:
    print("----------------------------------------------")
    print("***********************************************")
    print(k)
    print("***********************************************")
    print("----------------------------------------------")
    for x in characterDescription[k]:
        print(x)
        z=z+1
        print("----------------------------------------------")

----------------------------------------------
***********************************************
Privet Drive
***********************************************
----------------------------------------------
By the time The escape of the Brazilian boa constrictor was allowed out of The escape of the Brazilian boa constrictor cupboard again, the summer holidays had started and Dudley’s had already broken Dudley’s new video camera, crashed Dudley’s remote control airplane, and, first time out on Dudley’s racing bike, knocked down old Mrs. Figg as old Mrs. Figg crossed Privet Drive on old Mrs. Figg crutches.
----------------------------------------------
Potter The Cupboard under the Stairs Privet Drive Little Whinging Surrey The envelope was thick and heavy, made of yellowish parchment, and the address was written in emerald- green ink.
----------------------------------------------
Albus Dumbledore clicked the silver Put- Outer once, and twelve balls of light sped back to their street lamps 

----------------------------------------------
“Well — in that case ...” said Professor McGonagall, staring at the three of Harry and Ron, “Miss Granger, you foolish girl, how could you think of tackling a mountain troll on your own?” Hermione hung Hermione head.
----------------------------------------------
“We’ve got to close in on his broom!” Harry called, not taking Harry eyes off a large silver key that had a bent wing with the damaged wing.
----------------------------------------------
Harry pulled his eye out with difficulty, because his eye was very heavy, and, balancing his eye on Harry knee, let his eye fall open.
----------------------------------------------
Harry played with chessmen Seamus Finnigan had lent Harry, and old chessmen didn’t trust Harry at all.
----------------------------------------------
Harry pushed open the portrait of the Fat Lady and climbed through the portrait hole .
----------------------------------------------
“Weird).” Harry stared as Albus Dum

Uncle Vernon managed to say, “Go — cupboard — stay — no meals,” before Uncle Vernon collapsed into a chair, and His Aunt Petunia had to run and get Uncle Vernon a large brandy.
----------------------------------------------
Dudley’s had been accepted at Uncle Uncle Vernon’s old private school, Smeltings.
----------------------------------------------
An’ An’’s your bad luck you grew up in a family o’ the biggest Muggles I ever laid eyes on.” “we swore when we took him in we’d put a stop to that rubbish,” said Uncle Vernon, “swore we’d stamp it out of him!
----------------------------------------------
Tell they us don’t want — ” Harry could see Uncle Vernon’s shiny black shoes pacing up and down the kitchen.
----------------------------------------------
Uncle Vernon won’t pay for me to go and learn magic.” “Don’t worry about that,” said Hagrid, standing up and scratching his head.
----------------------------------------------
Three things lay on the doormat: a postcard from Uncle Ver

----------------------------------------------
Malfoy, Crabbe, and Goyle howled with laughter, but Ron, still not daring to take Ron eyes from the game, said, “You tell Ron, Neville.” Longbottom, if brains were gold you’d be poorer than Weasley, and thatFilch’s saying something.” Ron’s nerves were already stretched to the breaking point with anxiety about Harry.
----------------------------------------------
“Come on, Harry!” Hermione screamed, leaping onto her seat to watch as Harry sped straight at Snape — her didn’t even notice ” Malfoy and Ron rolling around under her seat, or the scuffles and yelps coming from the whirl of fists that was Neville, Crabbe, and Goyle.
----------------------------------------------
“I’ll get Crabbe,” said Ron, grinding Ron teeth at Malfoy’s back, “one of these days, I’ll get Crabbe — ” “I hate them both,” said Harry, “Malfoy and Snape.” “Come on, cheer up, it’s nearly Christmas,” said Hagrid.
----------------------------------------------
Scabbers the

In [306]:
with open("char_description15.txt", "w",encoding="utf-8") as f:
    for k in characterDescription:
        f.write("\n----------------------------------------------\n")
        f.write("\n***********************************************\n")
        f.write(k)
        f.write("\n***********************************************\n")
        f.write("\n----------------------------------------------\n")
        for x in characterDescription[k]:
            f.write(x)
            f.write("\n----------------------------------------------\n")

In [50]:
def apply_extraction(row):    
    doc=nlp(row)


    ## FIRST RULE OF DEPENDANCY PARSE -
    ## M - Sentiment modifier || A - Aspect
    ## RULE = M is child of A with a relationshio of amod
    rule1_pairs = []
    for token in doc:
        if token.dep_ == "amod":
            rule1_pairs.append((token.head.text, token.text))
            #return row['height'] * row['width']


    ## SECOND RULE OF DEPENDANCY PARSE -
    ## M - Sentiment modifier || A - Aspect
    #Direct Object - A is a child of something with relationship of nsubj, while
    # M is a child of the same something with relationship of dobj
    #Assumption - A verb will have only one NSUBJ and DOBJ

    rule2_pairs = []
    for token in doc:
        children = token.children
        A = "999999"
        M = "999999"
        for child in children :
            if(child.dep_ == "nsubj"):
                A = child.text
            if(child.dep_ == "dobj"):
                M = child.text
        if(A != "999999" and M != "999999"):
            rule2_pairs.append((A, M))


    ## THIRD RULE OF DEPENDANCY PARSE -
    ## M - Sentiment modifier || A - Aspect
    #Adjectival Complement - A is a child of something with relationship of nsubj, while
    # M is a child of the same something with relationship of acomp
    #Assumption - A verb will have only one NSUBJ and DOBJ

    rule3_pairs = []

    for token in doc:

        children = token.children
        A = "999999"
        M = "999999"
        for child in children :
            if(child.dep_ == "nsubj"):
                A = child.text

            if(child.dep_ == "acomp"):
                M = child.text

        if(A != "999999" and M != "999999"):
            rule3_pairs.append((A, M))

    ## FOURTH RULE OF DEPENDANCY PARSE -
    ## M - Sentiment modifier || A - Aspect

    #Adverbial modifier to a passive verb - A is a child of something with relationship of nsubjpass, while
    # M is a child of the same something with relationship of advmod

    #Assumption - A verb will have only one NSUBJ and DOBJ

    rule4_pairs = []
    for token in doc:


        children = token.children
        A = "999999"
        M = "999999"
        for child in children :
            if(child.dep_ == "nsubjpass"):
                A = child.text

            if(child.dep_ == "advmod"):
                M = child.text

        if(A != "999999" and M != "999999"):
            rule4_pairs.append((A, M))


    ## FIFTH RULE OF DEPENDANCY PARSE -
    ## M - Sentiment modifier || A - Aspect

    #Complement of a copular verb - A is a child of M with relationship of nsubj, while
    # M has a child with relationship of cop

    #Assumption - A verb will have only one NSUBJ and DOBJ

    rule5_pairs = []
    for token in doc:
        children = token.children
        A = "999999"
        buf_var = "999999"
        for child in children :
            if(child.dep_ == "nsubj"):
                A = child.text

            if(child.dep_ == "cop"):
                buf_var = child.text

        if(A != "999999" and buf_var != "999999"):
            rule3_pairs.append((A, token.text))

    aspects = []
    aspects = rule1_pairs + rule2_pairs + rule3_pairs +rule4_pairs +rule5_pairs
    dic = {"aspect_pairs" : aspects}
    return dic

In [51]:
noun_adj_pairs = defaultdict(list)

In [52]:
for k in characterDescription:
    print("----------------------------------------------")
    print("***********************************************")
    print(k)
    print("***********************************************")
    print("----------------------------------------------")
    for x in characterDescription[k]:
        print(apply_extraction(x))
        print("----------------------------------------------")

----------------------------------------------
***********************************************
Privet Drive
***********************************************
----------------------------------------------
{'aspect_pairs': [('constrictor', 'Brazilian'), ('constrictor', 'Brazilian'), ('camera', 'new'), ('control', 'remote'), ('time', 'first'), ('Figg', 'old'), ('Figg', 'old'), ('crutches', 'old'), ('Figg', 'Drive'), ('escape', 'again')]}
----------------------------------------------
{'aspect_pairs': [('parchment', 'yellowish'), ('ink', 'green'), ('Cupboard', 'Surrey'), ('envelope', 'thick')]}
----------------------------------------------
{'aspect_pairs': [('cat', 'tabby'), ('end', 'other'), ('Dumbledore', 'Outer'), ('Drive', 'orange'), ('Dumbledore', 'cat')]}
----------------------------------------------
{'aspect_pairs': [('moment', 'very'), ('voices', 'hushed'), ('step', 'front')]}
----------------------------------------------
{'aspect_pairs': [('corner', 'far')]}
--------------------

{'aspect_pairs': []}
----------------------------------------------
{'aspect_pairs': [('room', 'new')]}
----------------------------------------------
{'aspect_pairs': [('clothes', 'old'), ('presents', 'proper'), ('Harry’d', 'money'), ('Harry', 'Ron')]}
----------------------------------------------
{'aspect_pairs': [('toilet', 'poor'), ('anything', 'horrible'), ('toilet', 'anything'), ('Dudley’s’d', 'what'), ('it', 'sick')]}
----------------------------------------------
{'aspect_pairs': [('dial', 'lighted'), ('sofa', 'eaten'), ('wrist', 'fat'), ('watch', 'Harry'), ('he', 'eleven')]}
----------------------------------------------
{'aspect_pairs': [('morning', 'Next'), ('Petunia', 'hair')]}
----------------------------------------------
{'aspect_pairs': [('hotel', 'looking'), ('city', 'big')]}
----------------------------------------------
{'aspect_pairs': [('son', 'small'), ('boy', 'finer'), ('Dursley', 'son')]}
----------------------------------------------
{'aspect_pairs': [('fight'

----------------------------------------------
{'aspect_pairs': [('train', 'separately')]}
----------------------------------------------
{'aspect_pairs': []}
----------------------------------------------
{'aspect_pairs': []}
----------------------------------------------
{'aspect_pairs': [('Neville', 'white')]}
----------------------------------------------
{'aspect_pairs': [('an', 'more'), ('go', 'powerful')]}
----------------------------------------------
{'aspect_pairs': []}
----------------------------------------------
{'aspect_pairs': [('curse', 'deadly'), ('skin', 'bare'), ('pain', 'terrible'), ('chance', 'only'), ('pain', 'enough'), ('Quirrell', 'hand'), ('Harry', 'AAAARGH'), ('Quirrell', 'Quirrell'), ('Quirrell', 'blistering'), ('Quirrell', 'skin')]}
----------------------------------------------
{'aspect_pairs': [('fire', 'bright'), ('fire', 'blue'), ('Snape', 'fire'), ('that', 'around')]}
----------------------------------------------
{'aspect_pairs': [('I', 'wand')]}
----

{'aspect_pairs': [('you', 'attention'), ('Harry', 'Hermione')]}
----------------------------------------------
{'aspect_pairs': [('haired', 'red'), ('family', 'haired'), ('haired', 'red'), ('twins', 'haired'), ('Harry', 'family'), ('twins', 'what')]}
----------------------------------------------
{'aspect_pairs': [('something', 'big'), ('something', 'big'), ('something', 'squashy')]}
----------------------------------------------
{'aspect_pairs': []}
----------------------------------------------
{'aspect_pairs': [('study', 'more'), ('cauldron', 'solid'), ('set', 'nice'), ('telescope', 'collapsible'), ('couldn', 'curses'), ('yeh’ll', 'study'), ('Harry', 'level'), ('Harry', 'cauldron'), ('It', 'pewter'), ('they', 'set')]}
----------------------------------------------
{'aspect_pairs': []}
----------------------------------------------
{'aspect_pairs': [('case', 'bad'), ('nightmare', 'old'), ('figure', 'hooded'), ('Harry', 'case'), ('it', 'worse')]}
--------------------------------------

{'aspect_pairs': [('movement', 'sudden'), ('crate', 'large'), ('Harry', 'crate')]}
----------------------------------------------
{'aspect_pairs': [('years', 'first'), ('chamber', 'small'), ('chamber', 'empty'), ('Harry', 'drone'), ('McGonagall', 'years')]}
----------------------------------------------
{'aspect_pairs': [('three', 'other'), ('as', 'possible'), ('scurried', 'sweet')]}
----------------------------------------------
{'aspect_pairs': [('crowd', 'whole'), ('crowd', 'whole'), ('crowd', 'whole'), ('crowd', 'whole'), ('it', 'good')]}
----------------------------------------------
{'aspect_pairs': []}
----------------------------------------------
{'aspect_pairs': [('mouth', 'open')]}
----------------------------------------------
{'aspect_pairs': [('game', 'clean'), ('We', 'game'), ('Snape', 'excuse'), ('Harry', 'reason'), ('Harry', 'Quidditch')]}
----------------------------------------------
{'aspect_pairs': [('tongue', 'note')]}
---------------------------------------------

...

{'aspect_pairs': [('one', 'big')]}
----------------------------------------------
{'aspect_pairs': [('snowballs', 'several'), ('twins', 'Quirrell'), ('lake', 'solid')]}
----------------------------------------------
----------------------------------------------
***********************************************
Nicolas Flamel
***********************************************
----------------------------------------------
{'aspect_pairs': [('wizard', 'Dark'), ('I', 'you'), ('I', 'name'), ('I', 'name'), ('Dumbledore', 'famous')]}
----------------------------------------------
{'aspect_pairs': [('wizard', 'many'), ('wizard', 'greatest'), ('times', 'modern'), ('wizard', 'Dark'), ('Harry', 'card'), ('Dumbledore', 'famous')]}
----------------------------------------------
{'aspect_pairs': [('Ron', 'Hermione'), ('we', 'old')]}
----------------------------------------------
----------------------------------------------
***********************************************
Hermione
*********************

{'aspect_pairs': [('you', 'attention'), ('Harry', 'Hermione')]}
----------------------------------------------
{'aspect_pairs': [('Hermione', 'flat')]}
----------------------------------------------
{'aspect_pairs': [('Ridgebacks', 'Norwegian'), ('Hagrid', 'about')]}
----------------------------------------------
{'aspect_pairs': [('people', 'few'), ('you', 'Hermione'), ('Harry', 'Seamus')]}
----------------------------------------------
{'aspect_pairs': []}
----------------------------------------------
{'aspect_pairs': [('chamber', 'brilliantly'), ('chamber', 'lit'), ('sight', 'astonishing'), ('light', 'room')]}
----------------------------------------------
{'aspect_pairs': [('things', 'other'), ('Arts', 'Dark'), ('Snape', 'pocus')]}
----------------------------------------------
{'aspect_pairs': [('it', 'smiles'), ('we', 'you')]}
----------------------------------------------
{'aspect_pairs': [('chessmen', 'towering'), ('chessmen', 'white'), ('chessmen', 'faces')]}
----------------

{'aspect_pairs': [('teacher', 'nosed'), ('light', 'green'), ('Harry', 'turban'), ('it', 'heavier'), ('laugh', 'high')]}
----------------------------------------------
{'aspect_pairs': [('who', 'money'), ('you', 'brains'), ('Neville', 'red')]}
----------------------------------------------
{'aspect_pairs': [('Ron', 'nudged'), ('who', 'stunned')]}
----------------------------------------------
{'aspect_pairs': [('McGonagall', 'speechless')]}
----------------------------------------------
{'aspect_pairs': [('hand', 'other'), ('chance', 'big')]}
----------------------------------------------
{'aspect_pairs': [('night', 'last'), ('teeth', 'long'), ('it', 'staggerin'), ('I', 'Fang')]}
----------------------------------------------
{'aspect_pairs': [('years', 'first'), ('stories', 'long'), ('stories', 'boastful'), ('Malfoy', 'Muggles')]}
----------------------------------------------
{'aspect_pairs': []}
----------------------------------------------
{'aspect_pairs': []}
---------------------

{'aspect_pairs': [('package', 'small')]}
----------------------------------------------
{'aspect_pairs': [('lot', 'awful'), ('Harry', 'Snape')]}
----------------------------------------------
{'aspect_pairs': [('Hagrid', 'eyes'), ('Harry', 'that')]}
----------------------------------------------
{'aspect_pairs': [('I', 'match'), ('I', 'able'), ('I', 'lucky')]}
----------------------------------------------
{'aspect_pairs': [('reason', 'good'), ('number', 'extraordinary'), ('Harry’d', 'reason')]}
----------------------------------------------
{'aspect_pairs': [('faces', 'guilty'), ('something', 'eye')]}
----------------------------------------------
{'aspect_pairs': [('hairy', 'huge')]}
----------------------------------------------
{'aspect_pairs': [('Snape', 'Harry'), ('Snape', 'jot')]}
----------------------------------------------
{'aspect_pairs': [('room', 'common'), ('fires', 'roaring'), ('wind', 'bitter'), ('as', 'possible'), ('cauldrons', 'hot'), ('room', 'fires'), ('wind', 'win

{'aspect_pairs': [('Pig', 'you')]}
----------------------------------------------
{'aspect_pairs': [('Fat', 'something')]}
----------------------------------------------
{'aspect_pairs': [('we', 'portrait')]}
----------------------------------------------
{'aspect_pairs': []}
----------------------------------------------
{'aspect_pairs': [('three', 'other'), ('as', 'possible'), ('scurried', 'sweet')]}
----------------------------------------------
{'aspect_pairs': []}
----------------------------------------------
{'aspect_pairs': [('visit', 'nighttime')]}
----------------------------------------------
----------------------------------------------
***********************************************
Oliver Wood
***********************************************
----------------------------------------------
{'aspect_pairs': [('ball', 'bright'), ('ball', 'red'), ('Wood', 'ball')]}
----------------------------------------------
{'aspect_pairs': [('crate', 'large'), ('crate', 'wooden'), ('ball'

In [53]:
for k in characterDescription:
    for x in characterDescription[k]:
        noun_adj_pairs[k].append(apply_extraction(x))

In [63]:
pairs = defaultdict(set)

In [70]:
for j in characterDescription:
    for k in noun_adj_pairs[j]:
        for x in k:
            for l in k[x]:
                for z in lines_list:
                    x=re.search(z,l[0],re.IGNORECASE)
                    y=re.search(z,l[1],re.IGNORECASE)
                    if(x):
                        pairs[j].add(l)
                    if(y):
                        pairs[j].add(l)

In [72]:
for k in pairs:
    print(k)
    print("----------------------------------------")
    for x in pairs[k]:
        print(x)
    print("******************************************")

Privet Drive
----------------------------------------
('crutches', 'old')
('Figg', 'old')
('envelope', 'thick')
******************************************
Mr. Dursley
----------------------------------------
('he', 'eyes')
('man', 'beefy')
('clothes', 'funny')
('mouthed', 'open')
('order', 'large')
('boylived', 'large')
('mustache', 'large')
('stuff', 'Funny')
('daylight', 'broad')
('Dursley’d', 'legs')
('man', 'big')
('Dursley', 'thin')
('man', 'tiny')
('mouthed', 'gazed')
('people', 'young')
('lips', 'pursed')
('couple', 'young')
('Dudley', 'tantrum')
('report', 'fateful')
('man', 'old')
('doughnut', 'large')
('son', 'small')
******************************************
Dudley
----------------------------------------
('Petunia', 'tired')
('Dudley', 'everything')
('Dudley', 'anything')
('this', 'funny')
('eyes', 'small')
('Harry', 'glasses')
('Dudley', 'eyes')
('hotel', 'looking')
('puddin', 'fattenin')
('face', 'pink')
('head', 'fat')
('one', 'big')
('cobras', 'huge')
('snake', 'larges

('we', 'old')
('Magic', 'powerful')
('badge', 'red')
('feet', 'several')
('students', 'older')
******************************************
Sorcerer
----------------------------------------
('substance', 'legendary')
('Arts', 'Dark')
('book', 'old')
('things', 'other')
******************************************
Gringotts
----------------------------------------
('wizards', 'Dark')
******************************************
Quirrell
----------------------------------------
('scar', 'Harry')
('Arts', 'Dark')
('skin', 'bare')
('Quirrell', 'pale')
('pain', 'hot')
('they', 'large')
('nose', 'hooked')
('light', 'green')
('Force', 'Dark')
('Voldemort', 'shrieked')
('lake', 'solid')
('man', 'young')
('Harry', 'feet')
('Hermione', 'mouth')
('things', 'other')
('hair', 'black')
('Quirrell', 'small')
('It', 'dark')
('face', 'terrible')
('hole', 'small')
('smell', 'funny')
('Quirrell', 'lips')
('teacher', 'nosed')
('Quirrell', 'skin')
******************************************
Quidditch
------------

In [189]:
with open("description_noun_adj.txt", "w",encoding="utf-8") as f:
    f.write(str(noun_adj_pairs))

In [188]:
import json

json = json.dumps(noun_adj_pairs)
f = open("description_noun_adj.json","w")
f.write(json)
f.close()

In [218]:
lines_list = open('features.txt').read().splitlines()

In [219]:
lines_list

['thin',
 'hair',
 'height',
 'colour',
 'color',
 'eye',
 'eyes',
 'nose',
 'ear',
 'ears',
 'mouth',
 'lips',
 'neck',
 'lip',
 'eyebrows',
 'eyebrow',
 'glasses',
 'spectacles',
 'forehead',
 'scar',
 'tattoo',
 'arm',
 'arms',
 'short',
 'long',
 'fat',
 'tall',
 'body',
 'face',
 'lashes',
 'chin',
 'eyelashes',
 'eye lashes',
 'fat',
 'head',
 'jaw',
 'cheekk',
 'neck',
 'teeth',
 'tooth',
 'gums',
 'eyelash',
 'eyelid',
 'nostril',
 'iris',
 'beard',
 'mustache',
 'spectacles',
 'feet']

In [227]:
characterDescription3 = defaultdict(set)

In [228]:
def compareLists1(sentenceList1, majorCharacters1,k):
    """
    Compares the list of sentences with the character names and returns
    sentences that include names.
    """
    for sentence in sentenceList1:
        for name in majorCharacters1:
            #if re.search(name,sentence,re.IGNORECASE):
            if re.search(r"\b(?=\w)%s\b(?!\w)" % re.escape(name),
                         sentence,
                         re.IGNORECASE):
                characterDescription3[k].add(sentence)

In [229]:
for k in characterSentences:
    compareLists1(characterSentences[k],lines_list,k)

In [230]:
for k in characterDescription3:
    print("----------------------------------------------")
    print("***********************************************")
    print(k)
    print("***********************************************")
    print("----------------------------------------------")
    for x in characterDescription3[k]:
        print(x)
        z=z+1
        print("----------------------------------------------")

----------------------------------------------
***********************************************
Mr. Dursley
***********************************************
----------------------------------------------
Mr. Dursley was a big, beefy man with hardly any neck, although Mr. Dursley did have a very large mustache.
----------------------------------------------
Mr. Dursley was thin and blonde and had nearly twice the usual amount of neck, which came in very useful as Mr. Dursley spent so much of Mr. Dursley time craning over garden fences, spying on the neighbors.
----------------------------------------------
Mr. Dursley put the receiver back down and stroked Mr. Dursley mustache, thinking ...
----------------------------------------------
For a second, Mr. Dursley didn’t realize what Mr. Dursleyhad seen — then Mr. Dursley jerked Mr. Dursley head around to look again.
----------------------------------------------
Mr. Dursley drummed Mr. Dursley fingers on the steering wheel and Mr. Dursley 

“No post on Sundays,” Uncle Vernon reminded them cheerfully as Uncle Vernon spread marmalade on Uncle Vernon newspapers, “no damn letters today — ” Something came whizzing down the kitchen chimney as Something spoke and caught Something sharply on the back of the head.
----------------------------------------------
“Comb your hair!” Uncle Vernon barked, by way of a morning greeting.
----------------------------------------------
Harry thought this was strangely kind until Uncle Vernon stopped dead, facing the platforms with a nasty grin on Harry face.
----------------------------------------------
Uncle Vernon was holding a rifle in Uncle Vernon hands — now they knew what had been in the long, thin package Uncle Vernon had brought with they.
----------------------------------------------
Uncle Vernon won’t pay for me to go and learn magic.” “Don’t worry about that,” said Hagrid, standing up and scratching his head.
----------------------------------------------
------------------------

In [231]:
with open("description3_top15char.txt", "w",encoding="utf-8") as f:
    for k in characterDescription3:
        f.write("\n----------------------------------------------\n")
        f.write("\n***********************************************\n")
        f.write(k)
        f.write("\n***********************************************\n")
        f.write("\n----------------------------------------------\n")
        for x in characterDescription3[k]:
            f.write(x)
            f.write("\n----------------------------------------------\n")